In [1]:

import numpy as np
import pandas as pd
import tensorflow as tf
from training_utils import *
from scoring_func import get_scores
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from base_models import Model
from sklearn import preprocessing

W0718 07:04:17.589558 140562333595520 deprecation_wrapper.py:119] From /content/base_models.py:3: The name tf.layers.Dense is deprecated. Please use tf.compat.v1.layers.Dense instead.



In [0]:
'''
Config
'''
env = {
    "data_csv": "./happyworking.csv",
    "result_dir": "./result.csv"
}
args = {
    'batchsize': 256,
    'lr': 1e-4,
    'emb_size': 256,
    'dropout': 0.,
    'model': "widendeep",
    'log': "",
    'split_mode': "date", # or shuffle
    'n_bins': 100,
    'neg_sample_rate': 1,
    'n_epochs': 4,
    'n_hidden': 3,
    'dropout_type': 'none', # or 'none', 'field', 'embedding'
    'gpu': '0',
    'month': 'Jan'
}

# learning rate
LR = args['lr']
EMB_SIZE = args['emb_size']
DROPOUT_RATE = args['dropout']
N_BINS = args['n_bins']
NEG_SAMPLE_RATE = args['neg_sample_rate']
BATCHSIZE = args['batchsize']
N_EPOCHS = args['n_epochs']
N_HIDDEN = args['n_hidden']
DROPOUT_TYPE = args['dropout_type']
BASE_MODEL = args['model']
# log file
LOG = "logs/"+args['model']+'.json'
# path to save the model
saver_path ="saver/model"+args['model']
# dataset split mode
split_mode = args['split_mode']
# month
month = args['month']

#os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
#os.environ['CUDA_VISIBLE_DEVICES'] = args['gpu']

In [0]:
'''
preprocessing
'''
df = pd.read_csv(env['data_csv'],index_col=False)

lists = list(df['tradeDate'])
new_list=[]
for i in lists:
  new_list.append(i.replace('-',''))
df['tradeDate'] = new_list
df['tradeDate'] = df['tradeDate'].astype(np.int32)

df['rawticker'] = df['ticker']
df['ticker'] = df['ticker'].astype('category').cat.codes
df['exchangeCD']= df['exchangeCD'].astype('category').cat.codes


#normalization
unnorm_cols = ['lastChgPct','marketValue','negMarketValue',
             'PE','PB','FY12P','PCF','NetAssetGrowRate','VSTD10','avgTurnoverRate',
               'MACD','RSI','ROE','plusDI','minusDI','ADX','ASI','Hurst']

df[unnorm_cols] = df[unnorm_cols].apply(lambda x: (x - x.min())/(x.max() - x.min()))

In [0]:
'''
Split into train/valid/test.
'''
if split_mode=='date':  
    #valid = df[(20180700<df['tradeDate']) & (df['tradeDate']<20190000)]
    if month == 'Jan':
        train = df[df['tradeDate']<20180700]
        valid = df[(20180700<df['tradeDate']) & (df['tradeDate']<20190000)]
        test = df[(20190000<df['tradeDate']) & (df['tradeDate']<20190200)]
    elif month == 'Feb':
        train = df[(20160100<df['tradeDate']) & (df['tradeDate']<20180800)]
        valid = df[(20180800<df['tradeDate']) & (df['tradeDate']<20190200)]
        test = df[(20190200<df['tradeDate']) & (df['tradeDate']<20190300)]
    elif month == 'Mar':
        train = df[(20160200<df['tradeDate']) & (df['tradeDate']<20180900)]
        valid = df[(20180900<df['tradeDate']) & (df['tradeDate']<20190300)]
        test = df[(20190300<df['tradeDate']) & (df['tradeDate']<20190400)]
    elif month == 'Apr':
        train = df[(20160300<df['tradeDate']) & (df['tradeDate']<20181000)]
        valid = df[(20181000<df['tradeDate']) & (df['tradeDate']<20190400)]
        test = df[(20190400<df['tradeDate']) & (df['tradeDate']<20190500)]
    elif month == 'May':
        train = df[(20160400<df['tradeDate']) & (df['tradeDate']<20181100)]
        valid = df[(20181100<df['tradeDate']) & (df['tradeDate']<20190500)]
        test = df[(20190500<df['tradeDate']) & (df['tradeDate']<20190600)]
    elif month == 'Jun':
        train = df[(20160500<df['tradeDate']) & (df['tradeDate']<20181200)]
        valid = df[(20181200<df['tradeDate']) & (df['tradeDate']<20190600)]
        test = df[20190600<df['tradeDate']]

else:
    train = data[:int(0.6*len(data))]
    valid = data[int(0.6*len(data)):int(0.8*len(data))]
    test = data[int(0.8*len(data)):]

    


In [0]:
# specify useful columns of features
num_words_dict = {
    'ticker': 500,

    'exchangeCD':2,
    '01030101':2,
    '01030102':2,
    '01030103':2,'01030104':2,'01030105':2,'01030106':2,'01030107':2,'01030108':2,
    '01030109':2,'01030110':2,'01030111':2,'01030112':2,'01030113':2,'01030114':2,
    '01030116':2,'01030117':2,'01030118':2,'01030119':2,'01030301':2,'01030302':2,
    '01030303':2,'01030304':2,'01030305':2,'01030306':2,'01030307':2,'01030308':2,
    '01030309':2,'01030310':2,'01030311':2,'01030312':2,'01030313':2,'01030314':2,
    '01030315':2,'01030316':2,'01030317':2,'01030318':2,'01030319':2,'01030320':2,
    '01030321':2,'01030322':2,'01030323':2,'01030324':2,'01030325':2,'01030326':2,
    '01030327':2,'01030328':2,'01031400':2,'01031401':2,'01031402':2,'01031403':2,
    '01031404':2,'01031405':2,'01031406':2,'01031407':2,'01031408':2,'01031409':2
}
feat_cols = ['ticker','lastChgPct','marketValue','negMarketValue',
             'PE','PB','FY12P','PCF','NetAssetGrowRate','VSTD10','avgTurnoverRate',
             'MACD','RSI','ROE','plusDI','minusDI','ADX','ASI','Hurst','exchangeCD',

             
             '01030101','01030102',
             '01030103','01030104','01030105','01030106','01030107','01030108',
             '01030109','01030110','01030111','01030112','01030113','01030114',
             '01030116','01030117','01030118','01030119','01030301','01030302',
             '01030303','01030304','01030305','01030306','01030307','01030308',
             '01030309','01030310','01030311','01030312','01030313','01030314',
             '01030315','01030316','01030317','01030318','01030319','01030320',
             '01030321','01030322','01030323','01030324','01030325','01030326',
             '01030327','01030328','01031400','01031401','01031402','01031403',
             '01031404','01031405','01031406','01031407','01031408','01031409',
            ]

In [0]:
train_y = train['label'].astype(np.float32)
valid_y = valid['label'].astype(np.float32)
test_y = test['label'].astype(np.float32)

train_x = train[feat_cols]
valid_x = valid[feat_cols]
test_x = test[feat_cols]

In [7]:
model = Model(BASE_MODEL, feat_cols, num_words_dict,
              emb_size=EMB_SIZE, lr=LR, 
              dropout_rate=DROPOUT_RATE,
              n_hidden=N_HIDDEN, 
              embedding_dropout_type=DROPOUT_TYPE)

W0718 07:04:18.615402 140562333595520 deprecation_wrapper.py:119] From /content/base_models.py:193: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0718 07:04:18.621532 140562333595520 deprecation_wrapper.py:119] From /content/base_models.py:43: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0718 07:04:18.625141 140562333595520 deprecation_wrapper.py:119] From /content/base_models.py:45: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0718 07:04:18.627189 140562333595520 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0718 07:04:18.655598 140562333595520 depr

In [8]:
scores = {}
valid_predicts, test_predicts = {}, {}
"""
训练模型
"""
batchsize = int(NEG_SAMPLE_RATE*BATCHSIZE)
config = tf.ConfigProto() 
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())
for _ in range(N_EPOCHS):
    train_x_sampled, train_y_sampled = neg_sample(train_x, train_y, NEG_SAMPLE_RATE)
    train_on_batch(sess, model.train, train_x_sampled, train_y_sampled, batchsize, lr=LR, shuffle=False, verbose=True)
    # on training end
    valid_pred, valid_logit = predict_on_batch(sess, model.predict, valid_x)
    get_scores(valid_y, valid_pred, name='valid')
    valid_predicts['base'] = (valid_pred, valid_logit)

  0%|          | 0/55 [00:00<?, ?it/s]

valid	{'MSE': 0.01503807}


  0%|          | 0/55 [00:00<?, ?it/s]

valid	{'MSE': 0.01611981}


100%|██████████| 55/55 [00:12<00:00,  4.59it/s]


valid	{'MSE': 0.01581865}


In [9]:

test_pred, test_logit = predict_on_batch(sess, model.predict, test_x)
test_score = get_scores(test_y, test_pred, name='test')
scores = pd.DataFrame([test_score], index=['base'])
test_predicts['base'] = (test_pred, test_logit)


test	{'MSE': 0.01061565}


In [10]:
test_y.std()**2

0.007752903593044813

In [0]:

d = {'ticker':test['rawticker'],'date':test['tradeDate'],
     'actual':test['label'],'pred':test_pred}
dff = pd.DataFrame(data=d)
if month == 'Jan':
    dff.to_csv('result1.csv',index=False)
elif month == 'Feb':
    dff.to_csv('result2.csv',index=False)
elif month == 'Mar':
    dff.to_csv('result3.csv',index=False)
elif month == 'Apr':
    dff.to_csv('result4.csv',index=False)
elif month == 'May':
    dff.to_csv('result5.csv',index=False)
elif month == 'Jun':
    dff.to_csv('result6.csv',index=False)


In [12]:
'''
df1 = pd.read_csv('result1.csv')
df2 = pd.read_csv('result2.csv')
df3 = pd.read_csv('result3.csv')
df4 = pd.read_csv('result4.csv')
df5 = pd.read_csv('result5.csv')
df6 = pd.read_csv('result6.csv')
df = df1.append(df2,ignore_index=True)
df = df.append(df3,ignore_index=True)
df = df.append(df4,ignore_index=True)
df = df.append(df5,ignore_index=True)
df = df.append(df6,ignore_index=True)
df.to_csv('trial3.csv',index=False)
'''

"\ndf1 = pd.read_csv('result1.csv')\ndf2 = pd.read_csv('result2.csv')\ndf3 = pd.read_csv('result3.csv')\ndf4 = pd.read_csv('result4.csv')\ndf5 = pd.read_csv('result5.csv')\ndf6 = pd.read_csv('result6.csv')\ndf = df1.append(df2,ignore_index=True)\ndf = df.append(df3,ignore_index=True)\ndf = df.append(df4,ignore_index=True)\ndf = df.append(df5,ignore_index=True)\ndf = df.append(df6,ignore_index=True)\ndf.to_csv('trial3.csv',index=False)\n"